In [173]:
import pandas as pd
pd.set_option('max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

sns.set()

%matplotlib inline 


# import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
confusion_matrix, f1_score, roc_auc_score)
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [174]:
df_start = pd.read_csv('CCRB-Complaint-Data_202007271729/allegations_202007271729.csv')
df_start.head()

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,command_at_incident,rank_abbrev_incident,rank_abbrev_now,rank_now,rank_incident,mos_ethnicity,mos_gender,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
0,10004,Jonathan,Ruiz,078 PCT,8409,42835,7,2019,5,2020,078 PCT,POM,POM,Police Officer,Police Officer,Hispanic,M,32,Black,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions)
1,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,PBBS,POM,POM,Police Officer,Police Officer,White,M,24,Black,Male,26.0,Discourtesy,Action,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
2,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,PBBS,POM,POM,Police Officer,Police Officer,White,M,24,Black,Male,26.0,Offensive Language,Race,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
3,10007,John,Sears,078 PCT,5952,26146,7,2012,9,2013,PBBS,POM,POM,Police Officer,Police Officer,White,M,25,Black,Male,45.0,Abuse of Authority,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,Substantiated (Charges)
4,10009,Noemi,Sierra,078 PCT,24058,40253,8,2018,2,2019,078 PCT,POF,POF,Police Officer,Police Officer,Hispanic,F,39,NaN,NaN,16.0,Force,Physical force,67.0,Report-dispute,Arrest - other violation/crime,Substantiated (Command Discipline A)


In [175]:
print(df_start.shape)

(33358, 27)


In [176]:
df_start['year_received'].min()

1985

In [177]:
df_start['year_received'].max()

2020

In [178]:
df_start['fado_type'].value_counts()

Abuse of Authority    20292
Force                  7636
Discourtesy            4677
Offensive Language      753
Name: fado_type, dtype: int64

In [179]:
df_start['outcome_description'].value_counts()

No arrest made or summons issued        12822
Arrest - other violation/crime          10196
Summons - disorderly conduct             2118
Summons - other violation/crime          1940
Arrest - resisting arrest                1593
Arrest - disorderly conduct              1013
Arrest - assault (against a PO)           852
Moving violation summons issued           839
Arrest - OGA                              649
Other VTL violation summons issued        531
Parking summons issued                    279
Disorderly-Conduct/Arr/Summons            137
Arrest on Other Charge                     81
Traffic Summons Claimed or Issued          59
Juvenile Report                            57
Other Summons Claimed or Issued            38
Assault/Arrested                           34
Resisting Arrest/Arrested                  25
Arrest - harrassment (against a PO)        15
Obstruct-Govt-Admin/Arrested               10
Harrassment/Arrested/Summons                8
Summons - harrassment (against a P

In [180]:
df_start['board_disposition'].value_counts()

Unsubstantiated                             15448
Exonerated                                   9609
Substantiated (Charges)                      3796
Substantiated (Formalized Training)          1033
Substantiated (Command Discipline A)          964
Substantiated (Command Discipline)            851
Substantiated (Command Discipline B)          789
Substantiated (Command Lvl Instructions)      454
Substantiated (Instructions)                  248
Substantiated (No Recommendations)            165
Substantiated (MOS Unidentified)                1
Name: board_disposition, dtype: int64

In [181]:
df_start['outcome_description'].value_counts()

No arrest made or summons issued        12822
Arrest - other violation/crime          10196
Summons - disorderly conduct             2118
Summons - other violation/crime          1940
Arrest - resisting arrest                1593
Arrest - disorderly conduct              1013
Arrest - assault (against a PO)           852
Moving violation summons issued           839
Arrest - OGA                              649
Other VTL violation summons issued        531
Parking summons issued                    279
Disorderly-Conduct/Arr/Summons            137
Arrest on Other Charge                     81
Traffic Summons Claimed or Issued          59
Juvenile Report                            57
Other Summons Claimed or Issued            38
Assault/Arrested                           34
Resisting Arrest/Arrested                  25
Arrest - harrassment (against a PO)        15
Obstruct-Govt-Admin/Arrested               10
Harrassment/Arrested/Summons                8
Summons - harrassment (against a P

In [182]:
df_start['complainant_ethnicity'].value_counts()


Black              17114
Hispanic            6424
White               2783
Unknown             1041
Other Race           677
Asian                532
Refused              259
American Indian       64
Name: complainant_ethnicity, dtype: int64

In [183]:
columns = df_start.columns
columns


Index(['unique_mos_id', 'first_name', 'last_name', 'command_now', 'shield_no',
       'complaint_id', 'month_received', 'year_received', 'month_closed',
       'year_closed', 'command_at_incident', 'rank_abbrev_incident',
       'rank_abbrev_now', 'rank_now', 'rank_incident', 'mos_ethnicity',
       'mos_gender', 'mos_age_incident', 'complainant_ethnicity',
       'complainant_gender', 'complainant_age_incident', 'fado_type',
       'allegation', 'precinct', 'contact_reason', 'outcome_description',
       'board_disposition'],
      dtype='object')

In [184]:
dropped_columns = ['unique_mos_id', 'first_name', 'last_name', 'shield_no', 'complaint_id', 
                  'command_now', 'command_at_incident', 'rank_abbrev_incident', 'rank_abbrev_now', 'rank_now']
df = df_start.drop(dropped_columns, axis=1)
df.head(1)

,month_received,year_received,month_closed,year_closed,rank_incident,mos_ethnicity,mos_gender,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
0,7,2019,5,2020,Police Officer,Hispanic,M,32,Black,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions)


#### Keeping `contact_reason` and `outcome_description`  and `allegation` for now, but will have to figure out a way to narrow them down

In [185]:
df = pd.get_dummies(df, columns=['complainant_gender', 'mos_gender'], drop_first=True)

In [186]:
df= pd.get_dummies(df, columns=['complainant_ethnicity', 'mos_ethnicity', 'fado_type', 'board_disposition',
                                'rank_incident', ])

In [187]:
df.head()

,month_received,year_received,month_closed,year_closed,mos_age_incident,complainant_age_incident,allegation,precinct,contact_reason,outcome_description,complainant_gender_Gender non-conforming,complainant_gender_Male,complainant_gender_Not described,complainant_gender_Transman (FTM),complainant_gender_Transwoman (MTF),mos_gender_M,complainant_ethnicity_American Indian,complainant_ethnicity_Asian,complainant_ethnicity_Black,complainant_ethnicity_Hispanic,complainant_ethnicity_Other Race,complainant_ethnicity_Refused,complainant_ethnicity_Unknown,complainant_ethnicity_White,mos_ethnicity_American Indian,mos_ethnicity_Asian,mos_ethnicity_Black,mos_ethnicity_Hispanic,mos_ethnicity_White,fado_type_Abuse of Authority,fado_type_Discourtesy,fado_type_Force,fado_type_Offensive Language,board_disposition_Exonerated,board_disposition_Substantiated (Charges),board_disposition_Substantiated (Command Discipline A),board_disposition_Substantiated (Command Discipline B),board_disposition_Substantiated (Command Discipline),board_disposition_Substantiated (Command Lvl Instructions),board_disposition_Substantiated (Formalized Training),board_disposition_Substantiated (Instructions),board_disposition_Substantiated (MOS Unidentified),board_disposition_Substantiated (No Recommendations),board_disposition_Unsubstantiated,rank_incident_Captain,rank_incident_Chiefs and other ranks,rank_incident_Deputy Inspector,rank_incident_Detective,rank_incident_Inspector,rank_incident_Lieutenant,rank_incident_Police Officer,rank_incident_Sergeant
0,7,2019,5,2020,32,38.0,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,11,2011,8,2012,24,26.0,Action,67.0,Moving violation,Moving violation summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,11,2011,8,2012,24,26.0,Race,67.0,Moving violation,Moving violation summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,7,2012,9,2013,25,45.0,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,8,2018,2,2019,39,16.0,Physical force,67.0,Report-dispute,Arrest - other violation/crime,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [188]:
#Make all columns lower case 
df.columns = [x.lower() for x in df.columns]
#Sanity check 
df.head()

,month_received,year_received,month_closed,year_closed,mos_age_incident,complainant_age_incident,allegation,precinct,contact_reason,outcome_description,complainant_gender_gender non-conforming,complainant_gender_male,complainant_gender_not described,complainant_gender_transman (ftm),complainant_gender_transwoman (mtf),mos_gender_m,complainant_ethnicity_american indian,complainant_ethnicity_asian,complainant_ethnicity_black,complainant_ethnicity_hispanic,complainant_ethnicity_other race,complainant_ethnicity_refused,complainant_ethnicity_unknown,complainant_ethnicity_white,mos_ethnicity_american indian,mos_ethnicity_asian,mos_ethnicity_black,mos_ethnicity_hispanic,mos_ethnicity_white,fado_type_abuse of authority,fado_type_discourtesy,fado_type_force,fado_type_offensive language,board_disposition_exonerated,board_disposition_substantiated (charges),board_disposition_substantiated (command discipline a),board_disposition_substantiated (command discipline b),board_disposition_substantiated (command discipline),board_disposition_substantiated (command lvl instructions),board_disposition_substantiated (formalized training),board_disposition_substantiated (instructions),board_disposition_substantiated (mos unidentified),board_disposition_substantiated (no recommendations),board_disposition_unsubstantiated,rank_incident_captain,rank_incident_chiefs and other ranks,rank_incident_deputy inspector,rank_incident_detective,rank_incident_inspector,rank_incident_lieutenant,rank_incident_police officer,rank_incident_sergeant
0,7,2019,5,2020,32,38.0,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,11,2011,8,2012,24,26.0,Action,67.0,Moving violation,Moving violation summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,11,2011,8,2012,24,26.0,Race,67.0,Moving violation,Moving violation summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,7,2012,9,2013,25,45.0,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,8,2018,2,2019,39,16.0,Physical force,67.0,Report-dispute,Arrest - other violation/crime,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [189]:
#Create column for whether a complaint has been substantiated at all (encompus all substantiated categories)
condition = (df['board_disposition_substantiated (charges)']==1) | (df['board_disposition_substantiated (command discipline a)']==1) |(df['board_disposition_substantiated (command discipline b)']==1) |(df['board_disposition_substantiated (command discipline)'] ==1) |(df['board_disposition_substantiated (command lvl instructions)']==1) |(df['board_disposition_substantiated (formalized training)']==1) |(df['board_disposition_substantiated (instructions)']==1) |(df['board_disposition_substantiated (mos unidentified)']==1) |(df['board_disposition_substantiated (no recommendations)']==1)

df['substantiated_any'] = np.where(condition, 1, 0)

In [190]:
#Sanity check for new column 
df['substantiated_any'].value_counts()

0    25057
1     8301
Name: substantiated_any, dtype: int64

In [191]:
df.head()

,month_received,year_received,month_closed,year_closed,mos_age_incident,complainant_age_incident,allegation,precinct,contact_reason,outcome_description,complainant_gender_gender non-conforming,complainant_gender_male,complainant_gender_not described,complainant_gender_transman (ftm),complainant_gender_transwoman (mtf),mos_gender_m,complainant_ethnicity_american indian,complainant_ethnicity_asian,complainant_ethnicity_black,complainant_ethnicity_hispanic,complainant_ethnicity_other race,complainant_ethnicity_refused,complainant_ethnicity_unknown,complainant_ethnicity_white,mos_ethnicity_american indian,mos_ethnicity_asian,mos_ethnicity_black,mos_ethnicity_hispanic,mos_ethnicity_white,fado_type_abuse of authority,fado_type_discourtesy,fado_type_force,fado_type_offensive language,board_disposition_exonerated,board_disposition_substantiated (charges),board_disposition_substantiated (command discipline a),board_disposition_substantiated (command discipline b),board_disposition_substantiated (command discipline),board_disposition_substantiated (command lvl instructions),board_disposition_substantiated (formalized training),board_disposition_substantiated (instructions),board_disposition_substantiated (mos unidentified),board_disposition_substantiated (no recommendations),board_disposition_unsubstantiated,rank_incident_captain,rank_incident_chiefs and other ranks,rank_incident_deputy inspector,rank_incident_detective,rank_incident_inspector,rank_incident_lieutenant,rank_incident_police officer,rank_incident_sergeant,substantiated_any
0,7,2019,5,2020,32,38.0,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,11,2011,8,2012,24,26.0,Action,67.0,Moving violation,Moving violation summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,11,2011,8,2012,24,26.0,Race,67.0,Moving violation,Moving violation summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,7,2012,9,2013,25,45.0,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,8,2018,2,2019,39,16.0,Physical force,67.0,Report-dispute,Arrest - other violation/crime,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [192]:
df.dropna(subset=['precinct'], inplace=True)
df.isnull().sum()

month_received                                                   0
year_received                                                    0
month_closed                                                     0
year_closed                                                      0
mos_age_incident                                                 0
complainant_age_incident                                      4807
allegation                                                       1
precinct                                                         0
contact_reason                                                 199
outcome_description                                             56
complainant_gender_gender non-conforming                         0
complainant_gender_male                                          0
complainant_gender_not described                                 0
complainant_gender_transman (ftm)                                0
complainant_gender_transwoman (mtf)                           

In [197]:
df.duplicated().sum()

725

In [196]:
print(df.shape)

(33334, 53)


In [200]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [201]:
print(df.shape)

(32609, 53)


In [210]:
non_selected_features = ['board_disposition_substantiated (charges)', 'board_disposition_substantiated (command discipline a)', 'board_disposition_substantiated (command discipline b)',
                        'board_disposition_substantiated (command discipline)', 'board_disposition_substantiated (command lvl instructions)',
                        'board_disposition_substantiated (formalized training)', 'board_disposition_substantiated (instructions)',
                        'board_disposition_substantiated (mos unidentified)','board_disposition_substantiated (no recommendations)', 
                        'substantiated_any', 'board_disposition_exonerated', 'board_disposition_unsubstantiated', 'contact_reason', 'outcome_description', 'allegation', 'complainant_age_incident']
selected_features = df.columns.drop(non_selected_features)
selected_features

Index(['month_received', 'year_received', 'month_closed', 'year_closed',
       'mos_age_incident', 'precinct',
       'complainant_gender_gender non-conforming', 'complainant_gender_male',
       'complainant_gender_not described', 'complainant_gender_transman (ftm)',
       'complainant_gender_transwoman (mtf)', 'mos_gender_m',
       'complainant_ethnicity_american indian', 'complainant_ethnicity_asian',
       'complainant_ethnicity_black', 'complainant_ethnicity_hispanic',
       'complainant_ethnicity_other race', 'complainant_ethnicity_refused',
       'complainant_ethnicity_unknown', 'complainant_ethnicity_white',
       'mos_ethnicity_american indian', 'mos_ethnicity_asian',
       'mos_ethnicity_black', 'mos_ethnicity_hispanic', 'mos_ethnicity_white',
       'fado_type_abuse of authority', 'fado_type_discourtesy',
       'fado_type_force', 'fado_type_offensive language',
       'rank_incident_captain', 'rank_incident_chiefs and other ranks',
       'rank_incident_deputy inspe

In [211]:
X = df[selected_features]
y = df['substantiated_any']

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
print('Length of our Training data:', X_train.shape, '\nLength of our Testing data:', X_test.shape)

Length of our Training data: (26087, 37) 
Length of our Testing data: (6522, 37)


In [213]:
params = {'criterion': ['gini', 'entropy'], 
         'max_depth': [2, 4, 5, 10, 20, 100], 
         'min_samples_split': [2, 20, 100], 
         'min_samples_leaf': [1, 2, 10, 20, 100]
         }
grid_search_cv =  GridSearchCV( 
    estimator=DecisionTreeClassifier(), 
    param_grid = params, 
    scoring = 'f1'
    )

In [214]:
grid_search_cv.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 5, 10, 20, 100],
                         'min_samples_leaf': [1, 2, 10, 20, 100],
                         'min_samples_split': [2, 20, 100]},
             scoring='f1')

In [215]:
print( grid_search_cv.best_params_ )

{'criterion': 'gini', 'max_depth': 100, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [216]:
# 1. Evaluate your best model using accuracy, precision, recall, f1 scores, and auc scores. 

# This command gives you the best tree
model = grid_search_cv.best_estimator_

# Now lets evaluate our model
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy Score: %f" % accuracy)

precision = precision_score(y_true=y_test, y_pred=y_pred)
print("Precision Score: %f" % precision)

recall = recall_score(y_true=y_test, y_pred=y_pred)
print("Recall Score: %f" % recall)

f1 = f1_score(y_true=y_test, y_pred=y_pred)
print('F1 Score: %f' % f1)

# Calculate predicted probabilities, keep only probability for when class = 1
y_pred_proba = model.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
print('AUC Score: %f' % auc)

Accuracy Score: 0.738117
Precision Score: 0.492455
Recall Score: 0.425860
F1 Score: 0.456743
AUC Score: 0.671562


In [217]:
# 1. Which feature was your most important feature?
# Now lets look at our feature importances
feature_imp = pd.DataFrame.from_dict( {'feature_importance': model.feature_importances_,
                                       'feature':selected_features }).sort_values('feature_importance', ascending=False)
feature_imp

,feature_importance,feature
5,0.189367,precinct
4,0.149594,mos_age_incident
0,0.130496,month_received
2,0.127548,month_closed
3,0.069167,year_closed
1,0.063124,year_received
25,0.032475,fado_type_abuse of authority
14,0.023267,complainant_ethnicity_black
7,0.021609,complainant_gender_male
15,0.020860,complainant_ethnicity_hispanic


In [218]:
params_rf = {'criterion': ['gini', 'entropy'], 
         'max_depth': [2, 4, 5, 10, 20, 100], 
         'max_features': [2, 5, 10, 15, 25, 'auto'], 
         'min_samples_split': [2, 10, 20, 100], 
         'n_estimators': [2, 5, 10, 20, 100]
         }
grid_search_cv_rf =  GridSearchCV( 
    estimator=RandomForestClassifier(), 
    param_grid = params_rf, 
    scoring = 'f1'
    )

In [219]:
grid_search_cv_rf.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 5, 10, 20, 100],
                         'max_features': [2, 5, 10, 15, 25, 'auto'],
                         'min_samples_split': [2, 10, 20, 100],
                         'n_estimators': [2, 5, 10, 20, 100]},
             scoring='f1')

In [221]:
print( grid_search_cv_rf.best_params_ )

{'criterion': 'gini', 'max_depth': 100, 'max_features': 25, 'min_samples_split': 2, 'n_estimators': 20}


In [226]:
# 1. Evaluate your best model using accuracy, precision, recall, f1 scores, and auc scores. 

# This command gives you the best tree
model_rf = grid_search_cv_rf.best_estimator_

# Now lets evaluate our model
y_pred_rf = model_rf.predict(X_test)

accuracy_rf = accuracy_score(y_true=y_test, y_pred=y_pred_rf)
print("Accuracy Score: %f" % accuracy_rf)

precision_rf = precision_score(y_true=y_test, y_pred=y_pred_rf)
print("Precision Score: %f" % precision_rf)

recall_rf = recall_score(y_true=y_test, y_pred=y_pred_rf)
print("Recall Score: %f" % recall_rf)

f1_rf = f1_score(y_true=y_test, y_pred=y_pred_rf)
print('F1 Score: %f' % f1_rf)

# Calculate predicted probabilities, keep only probability for when class = 1
y_pred_proba_rf = model_rf.predict_proba(X_test)[:,1]
auc_rf = roc_auc_score(y_true=y_test, y_score=y_pred_proba_rf)
print(f'AUC Score: %f' % auc_rf)

Accuracy Score: 0.758663
Precision Score: 0.547619
Recall Score: 0.381969
F1 Score: 0.450035
AUC Score: 0.743384


In [227]:
feature_imp_rf = pd.DataFrame.from_dict( {'feature_importance': model_rf.feature_importances_,
                                       'feature':selected_features }).sort_values('feature_importance', ascending=False)
feature_imp_rf

,feature_importance,feature
5,0.190244,precinct
4,0.154848,mos_age_incident
2,0.120853,month_closed
0,0.120253,month_received
3,0.072210,year_closed
1,0.070284,year_received
25,0.024759,fado_type_abuse of authority
14,0.022510,complainant_ethnicity_black
24,0.021658,mos_ethnicity_white
7,0.020383,complainant_gender_male
